In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def huber_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    epsilon=1.35,
    alpha=0.0001,
    max_iter=1000,
    use_cv=False,
    cv_params=None,
    random_state=None
):
    """
    Robust Huber regression for time series forecasting with:
    - Automatic hyperparameter tuning via grid search
    - Outlier-resistant modeling
    - Enhanced differencing checks
    """
    # Create directory structure
    os.makedirs('HuberRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing (fixed adf_result check)
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:  # Fixed p-value access
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features with type consistency
    df = pd.DataFrame({'y': series.astype(np.float64)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(np.float64)
    df = df.dropna()

    # 3. Time-based split with validation
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model training with optional CV
    if use_cv:
        # Default parameter grid
        param_grid = cv_params or {
            'epsilon': [1.0, 1.35, 1.5, 2.0],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
        
        grid_search = GridSearchCV(
            HuberRegressor(max_iter=max_iter),
            param_grid,
            cv=5,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_epsilon = model.epsilon
        best_alpha = model.alpha
    else:
        model = HuberRegressor(
            epsilon=epsilon,
            alpha=alpha,
            max_iter=max_iter
        )
        model.fit(X_train, y_train)
        best_epsilon = epsilon
        best_alpha = alpha

    # 5. Forecasting with type consistency
    pred_test = model.predict(X_test).astype(np.float64)

    # 6. Inverse differencing with edge case handling
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Enhanced DataFrame with confidence intervals
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test,
        'residual': y_test.values - pred_test
    })

    # 8. Robust CSV saving
    forecast_csv_path = os.path.join('HuberRegression', 'huber_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )
     # 9. Comprehensive metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'final_epsilon': best_epsilon,
        'final_alpha': best_alpha,
        'differencing': d,
        'n_iterations': model.n_iter_
    }

    if use_cv:
        metrics_data.update({
            'best_params': grid_search.best_params_,
            'cv_results': grid_search.cv_results_
        })

    metrics_df = pd.DataFrame([metrics_data])
    
    metrics_csv_path = os.path.join('HuberRegression', 'huber_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 10. Enhanced plotting with residuals
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Forecast plot
    ax1.plot(original_series.index, original_series.values, label='Original Series')
    ax1.plot(forecast_df['date'], forecast_df['forecast'], 
            label='Huber Forecast', color='darkorange')
    ax1.set_title(f'Huber Regression Forecast for {district_name}\nRMSE: {rmse:.2f}')
    ax1.legend()

    # Residual plot
    ax2.bar(forecast_df['date'], forecast_df['residual'], 
           color='gray', alpha=0.7, width=5)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_title('Forecast Residuals')
    
    plt.tight_layout()
    plt.savefig(os.path.join('HuberRegression', f'huber_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_params': model.get_params(),
        'feature_names': X_train.columns.tolist()
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I8"].asfreq('MS')
        
        results = huber_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'epsilon': [1.0, 1.35, 1.7],
                'alpha': [0.0001, 0.001, 0.01]
            },
            max_iter=2000
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district         rmse  r_squared  \
0  AHMEDNAGAR  1816.157364   0.142524   

                                        coefficients   intercept  \
0  [-0.40130591518351016, -0.20590670812999645, -...  159.751747   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             1            55   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.01862163543701172, 0.0097...  

=== Forecast Data ===
     district       date  actual     forecast     residual
0  AHMEDNAGAR 2020-07-01    5982  6568.228236  -586.228236
1  AHMEDNAGAR 2020-08-01    5559  6992.569382 -1433.569382
2  AHMEDNAGAR 2020-09-01    5903  7488.001266 -1585.001266
3  AHMEDNAGAR 2020-10-01    5719  7649.723720 -1930.723720
4  AHMEDNAGAR 2020-11-01    5427  7869.883218 -2442.883218
=== Metrics ===
  district        rmse  r_squared  \
0    AKOLA  269.235765  

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
2 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
     district        rmse  r_squared  \
0  CHANDRAPUR  716.404472   0.047055   

                                        coefficients  intercept  \
0  [0.06179570159368598, -0.14093020965822556, 0.... -31.269091   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0       0.0001             1            54   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.014848566055297852, 0.009...  

=== Forecast Data ===
     district       date  actual     forecast    residual
0  CHANDRAPUR 2020-07-01    2279  2350.399615  -71.399615
1  CHANDRAPUR 2020-08-01    2103  2297.780554 -194.780554
2  CHANDRAPUR 2020-09-01    2030  2215.901389 -185.901389
3  CHANDRAPUR 2020-10-01    2073  2194.841201 -121.841201
4  CHANDRAPUR 2020-11-01    1897  2144.249301 -247.249301
=== Metrics ===
  district        rmse  r_squared  \
0    DHULE  762.604716   0.205

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district        rmse  r_squared  \
0    LATUR  791.947125   0.095416   

                                        coefficients   intercept  \
0  [-0.24990296972565357, -0.1872602856833662, -0...  105.744967   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             1            61   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.0137603759765625, 0.00750...  

=== Forecast Data ===
  district       date  actual     forecast     residual
0    LATUR 2020-07-01    3500  3808.957528  -308.957528
1    LATUR 2020-08-01    3274  3974.416690  -700.416690
2    LATUR 2020-09-01    3634  4200.300185  -566.300185
3    LATUR 2020-10-01    3408  4299.732912  -891.732912
4    LATUR 2020-11-01    3363  4427.433208 -1064.433208
=== Metrics ===
  district        rmse  r_squared  \
0   MUMBAI  648.421477  -0.074095   

            

...

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
    district         rmse  r_squared  \
0  NANDURBAR  1076.634301    0.25594   

                                        coefficients  intercept  \
0  [-0.33946551461137014, 0.268136601549362, 0.18...   9.398548   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.0        0.001             1            35   

                        best_params  \
0  {'alpha': 0.001, 'epsilon': 1.0}   

                                          cv_results  
0  {'mean_fit_time': [0.013193798065185548, 0.004...  

=== Forecast Data ===
    district       date  actual     forecast     residual
0  NANDURBAR 2020-07-01    3193  3641.963803  -448.963803
1  NANDURBAR 2020-08-01    2585  3806.961684 -1221.961684
2  NANDURBAR 2020-09-01    2762  3937.517052 -1175.517052
3  NANDURBAR 2020-10-01    2322  3673.554462 -1351.554462
4  NANDURBAR 2020-11-01    2311  3766.625413 -1455.625413
=== Metrics ===
  district        rmse  r_squared  \
0   NASHIK  473.306312   0.00212

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
  district        rmse  r_squared  \
0   SANGLI  324.848163   0.140158   

                                        coefficients  intercept  \
0  [-0.30050772566601597, -0.11475445854667292, 0...  35.700168   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7        0.001             1            56   

                        best_params  \
0  {'alpha': 0.001, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.014856195449829102, 0.006...  

=== Forecast Data ===
  district       date  actual     forecast    residual
0   SANGLI 2020-07-01    1914  1780.195279  133.804721
1   SANGLI 2020-08-01    1544  1680.285929 -136.285929
2   SANGLI 2020-09-01    1534  1832.551041 -298.551041
3   SANGLI 2020-10-01    1492  1946.400767 -454.400767
4   SANGLI 2020-11-01    1581  1932.219424 -351.219424
=== Metrics ===
  district         rmse  r_squared  \
0   SATARA  2340.472841   0.089768   

                

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual     forecast    residual
 0  MUMBAI SUBURBAN 2021-02-01    9380  9162.645284  217.354716
 1  MUMBAI SUBURBAN 2021-03-01    9076  9499.263891 -423.263891,
 'metrics_df':           district        rmse  r_squared  \
 0  MUMBAI SUBURBAN  336.448654  -0.292176   
 
                                         coefficients  intercept  \
 0  [0.11129439962905885, -0.21419495254903959, 0....   0.007577   
 
    final_epsilon  final_alpha  differencing  n_iterations  \
 0            1.0         0.01             1            22   
 
                        best_params  \
 0  {'alpha': 0.01, 'epsilon': 1.0}   
 
                                           cv_results  
 0  {'mean_fit_time': [0.013492059707641602, 0.007...  ,
 'model_params': {'alpha': 0.01,
  'epsilon': 1.0,
  'fit_intercept': True,
  'max_iter': 2000,
  'tol': 1e-05,
  'warm_start': False},
 'feature_names': ['lag_1', 'lag_2', 'lag_3']}